In [1]:
import os

In [2]:
os.getcwd()

'C:\\Misc\\GIT\\DLNLPProject\\notebooks'

In [3]:
os.chdir("..")

In [4]:
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier

In [5]:
import pandas as pd

In [31]:
import pickle

In [6]:
# Download the Romanian language model for nltk
nltk.download('punkt', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words', 'rslp', 'universal_tagset')

True

In [7]:
train_df = pd.read_csv('data/train_sanitized.csv')

In [8]:
test_df  = pd.read_csv('data/test_sanitized.csv')

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29651 entries, 0 to 29650
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    29651 non-null  int64  
 1   text     29637 non-null  object 
 2   label    29651 non-null  float64
 3   dataset  29651 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 926.7+ KB


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23005 entries, 0 to 23004
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  23005 non-null  int64  
 1   text        23001 non-null  object 
 2   label       23005 non-null  float64
 3   dataset     23005 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 719.0+ KB


In [11]:
dataset = train_df.loc[:, ['text', 'label']]

In [12]:
dataset_test = test_df.loc[:, ['text', 'label']]

In [13]:
dataset

,text,label
0,slab aspect frumoasa vrea juca tastatură,0.0
1,multumite incarcă repede incarcar complet telefon,1.0
2,teapa zi mare rand stica spune producator plas...,0.0
3,merita cumparat merita cumpara creat limitare ...,0.0
4,recomanda ceas excelent nevoie vreme nu el com...,1.0
...,...,...
29646,grija notificar cumparat telefon venit android...,0.0
29647,sotie plăcea inchiriem film groaznic sci-fi pr...,0.0
29648,stiu puteti calificativ stea film macar traile...,0.0
29649,tradare veni insasi citire ceatie vedea intrea...,0.0


In [17]:
dataset_test

,text,label
0,slab aspect frumoasa vrea juca tastatură,0.0
1,multumite incarcă repede incarcar complet telefon,1.0
2,teapa zi mare rand stica spune producator plas...,0.0
3,merita cumparat merita cumpara creat limitare ...,0.0
4,recomanda ceas excelent nevoie vreme nu el com...,1.0
...,...,...
23000,film contin efect sonor gs trebui impusca exis...,0.0
23001,preluare imperi roman vizionati miniserle exce...,0.0
23002,eu avea place film inchiriat el speranta asema...,0.0
23003,singur parte film sfarsit st parte film apri l...,0.0


In [15]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bogdan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
# Convert the DataFrame
reviews = []
stopwords = nltk.corpus.stopwords.words('romanian')
for _, row in dataset.iterrows():
    review = row['text']
    label = row['label']
    try:
        review_tokens = [w for w in nltk.word_tokenize(review) if not w in stopwords]
        reviews.append((review_tokens, label))
    except:
        pass

In [19]:
reviews

[(['slab', 'aspect', 'frumoasa', 'vrea', 'juca', 'tastatură'], 0.0),
 (['multumite', 'incarcă', 'repede', 'incarcar', 'complet', 'telefon'], 1.0),
 (['teapa',
   'mare',
   'rand',
   'stica',
   'spune',
   'producator',
   'plastic',
   'calitate',
   'veni',
   'gata',
   'zgariata',
   'lipest',
   'ecran',
   'formă',
   'urma',
   'atent',
   'instructiune',
   'aveti',
   'comandati'],
  0.0),
 (['merita',
   'cumparat',
   'merita',
   'cumpara',
   'creat',
   'limitare',
   'volum',
   'volum',
   'jumate',
   'comparatie',
   'casti',
   'ieftin',
   'trebui',
   'returnez'],
  0.0),
 (['recomanda',
   'ceas',
   'excelent',
   'nevoie',
   'vreme',
   'compar',
   'telefon',
   'baterie',
   'baterie',
   'folosi',
   'inten',
   'primesti',
   'notificarile',
   'timp',
   'real',
   'blochează',
   'comod',
   'arată',
   'premium',
   'comparatia',
   'ceas',
   'smart',
   'baterie',
   'categorie',
   'adreseaza',
   'extensie',
   'telefon'],
  1.0),
 (['bun',
   'pic

In [21]:
# Convert the test DataFrame
reviews_test = []
stopwords = nltk.corpus.stopwords.words('romanian')
for _, row in dataset_test.iterrows():
    review = row['text']
    label = row['label']
    try:
        review_tokens = [w for w in nltk.word_tokenize(review) if not w in stopwords]
        reviews_test.append((review_tokens, label))
    except:
        pass

In [22]:
#frequency distribution of the words in the reviews
all_words = []
for review, label in reviews:
    all_words += review
fdist = nltk.FreqDist(all_words)

In [23]:
#select the most informative words
word_features = list(fdist.keys())[:3000]

In [51]:
word_features[:30]

['slab',
 'aspect',
 'frumoasa',
 'vrea',
 'juca',
 'tastatură',
 'multumite',
 'incarcă',
 'repede',
 'incarcar',
 'complet',
 'telefon',
 'teapa',
 'mare',
 'rand',
 'stica',
 'spune',
 'producator',
 'plastic',
 'calitate',
 'veni',
 'gata',
 'zgariata',
 'lipest',
 'ecran',
 'formă',
 'urma',
 'atent',
 'instructiune',
 'aveti']

In [25]:
#features for each review
def extract_features(review):
    review_words = set(review)
    features = {}
    for word in word_features:
        features[word] = (word in review_words)
    return features

In [26]:
#Bayes classifier
featuresets = [(extract_features(review), label) for review, label in reviews]
classifierNLTKnb = nltk.NaiveBayesClassifier.train(featuresets)

In [52]:
featuresets_test = [(extract_features(review), label) for review, label in reviews_test]

[({'slab': True,
   'aspect': True,
   'frumoasa': True,
   'vrea': True,
   'juca': True,
   'tastatură': True,
   'multumite': False,
   'incarcă': False,
   'repede': False,
   'incarcar': False,
   'complet': False,
   'telefon': False,
   'teapa': False,
   'mare': False,
   'rand': False,
   'stica': False,
   'spune': False,
   'producator': False,
   'plastic': False,
   'calitate': False,
   'veni': False,
   'gata': False,
   'zgariata': False,
   'lipest': False,
   'ecran': False,
   'formă': False,
   'urma': False,
   'atent': False,
   'instructiune': False,
   'aveti': False,
   'comandati': False,
   'merita': False,
   'cumparat': False,
   'cumpara': False,
   'creat': False,
   'limitare': False,
   'volum': False,
   'jumate': False,
   'comparatie': False,
   'casti': False,
   'ieftin': False,
   'trebui': False,
   'returnez': False,
   'recomanda': False,
   'ceas': False,
   'excelent': False,
   'nevoie': False,
   'vreme': False,
   'compar': False,
   'bate

In [28]:
# Test
review = 'Acesta este un review pozitiv'
review_tokens = [w for w in nltk.word_tokenize(review) if not w in stopwords]
prob_dist = classifierNLTKnb.prob_classify(extract_features(review_tokens))
predicted_label = prob_dist.max()
print('Predicted label:', predicted_label)

Predicted label: 1.0


In [29]:
accuracy = nltk.classify.accuracy(classifierNLTKnb, featuresets_test)

In [30]:
accuracy

0.6694056780139994

In [48]:
classifierNLTKnb.show_most_informative_features()

Most Informative Features
             dezamagitor = True             0.25 : 1.0    =    214.5 : 1.0
                   retur = True             0.25 : 1.0    =    112.7 : 1.0
           calitate-pret = True             0.75 : 0.0    =    107.0 : 1.0
                 returna = True             0.25 : 1.0    =     89.0 : 1.0
             nemultumite = True              0.0 : 1.0    =     85.7 : 1.0
                   teapa = True              0.0 : 1.0    =     80.7 : 1.0
               nemultumi = True              0.0 : 1.0    =     79.0 : 1.0
                    film = True              0.0 : 0.75   =     60.0 : 1.0
               rezonabil = True             0.75 : 0.0    =     56.0 : 1.0
                excelent = True              1.0 : 0.25   =     54.6 : 1.0


In [32]:
#save model
f = open('data/BayesSanitizedText.pickle', 'wb')
pickle.dump(classifierNLTKnb, f)
f.close()

In [33]:
#load model
f = open('data/BayesSanitizedText.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [34]:
#verify loaded model acc
accuracy = nltk.classify.accuracy(classifier, featuresets_test)
accuracy

0.6694056780139994